<a href="https://colab.research.google.com/github/richukuttan/AccelerometerGraph/blob/master/model/lstm2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import os
import glob
import csv
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
!pip install funcy
import funcy
from funcy import print_durations
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from torch.cuda.amp import GradScaler
torch.backends.cudnn.benchmark = True
import datetime
import gc
import pickle

In [17]:
start_date = '1980-01-01'

In [18]:
if not os.path.isdir('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive')

In [19]:
macro_df = pd.read_csv('/content/drive/MyDrive/11-785/Project/macro_features_final_dropna.csv',index_col=0).loc[start_date:]
macro_df = macro_df.dropna(axis=0,how='all')
macro_df = macro_df.dropna(axis=1,how='all')
# features_idx = macro_df.columns
macro_df

,W875RX1,DPCERA3M086SBEA,CMRMTSPL,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,IPNCONGD,IPBUSEQ,...,RNA,ROA,ROE,SGA2S,D2A,AC,OA,OL,PCM,AT
rdq,,,,,,,,,,,,,,,,,,,,,
1984-03-31,5542.4,43.313,625030.688,53.7313,59.7733,58.3060,70.0584,49.1906,79.4306,32.2755,...,0.094449,0.050187,0.117328,0.681474,0.037466,0.001090,-0.000767,1.091945,-1.399036,91.9985
1984-06-30,5644.3,44.151,639093.221,54.5095,60.5822,58.9702,69.9638,48.8344,79.5199,33.0531,...,0.100075,0.053498,0.122086,0.673583,0.037448,0.000000,-0.000824,1.096514,-1.340794,87.9295
1985-08-31,5845.1,46.763,659042.751,54.8269,62.0818,60.5334,70.5367,49.1804,80.1992,34.4353,...,0.092896,0.047344,0.130653,0.693678,0.038749,-0.003140,-0.001191,1.095180,-1.258783,80.6030
1985-11-30,5910.7,46.791,659509.852,55.0381,62.5036,61.0456,71.2028,50.0607,80.7057,34.5959,...,0.090193,0.044600,0.127164,0.700228,0.039440,0.000000,-0.001074,1.076283,-1.265303,80.7480
1986-05-31,6043.6,47.980,669565.217,55.2556,63.0492,61.2452,72.7183,51.0468,82.4684,33.9200,...,0.084047,0.039336,0.134048,0.710053,0.039774,0.000000,-0.001109,1.075452,-1.252150,80.6865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-30,14406.3,125.640,1572476.000,102.0374,101.2518,101.6865,100.8504,104.7309,99.7321,95.8557,...,0.049653,0.013872,0.069816,0.659218,0.022986,0.003672,-0.000196,0.474068,-0.202465,1124.9105
2021-12-31,14398.0,123.868,1562935.000,101.6402,100.7942,101.2701,100.2659,103.9276,99.2067,95.7659,...,0.049773,0.013892,0.069949,0.659782,0.022915,0.004113,-0.000202,0.475670,-0.205471,1131.1950
2022-01-31,14363.1,126.504,1594543.000,102.6863,101.8134,102.4001,101.8697,106.0272,100.6766,95.4291,...,0.049706,0.013879,0.070214,0.659905,0.022857,0.003672,-0.000198,0.478045,-0.212693,1136.5500


In [20]:
returns_df = pd.read_csv('/content/drive/MyDrive/11-785/Project/returns.csv',index_col=0).loc[start_date:]
returns_df = returns_df.dropna(axis=0,how='all')
returns_df = returns_df.dropna(axis=1,how='all')
returns_df.clip(returns_df.quantile(0.01,axis=1),returns_df.quantile(0.99,axis=1),axis=0)
date_idx = returns_df.index
equity_idx = returns_df.columns
returns_df

,0081A,0091A,0099A,0127A,0138A,0146A,0153A,0183B,0223B,0230B,...,ZVXI,ZWRK,ZWS,ZXAIY,ZY,ZYME,ZYNE,ZYXI,ZZ,ZZ1
datadate,,,,,,,,,,,,,,,,,,,,,
1980-01-31,NaN,0.012579,NaN,0.068182,0.039548,NaN,NaN,NaN,0.007752,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.137500
1980-02-29,NaN,-0.006211,NaN,0.000000,NaN,NaN,NaN,NaN,-0.086462,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.164835
1980-03-31,NaN,-0.306250,NaN,0.007092,NaN,NaN,NaN,NaN,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.118421
1980-04-30,NaN,0.252252,NaN,-0.126761,NaN,NaN,NaN,NaN,0.139130,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.029851
1980-05-31,NaN,0.071942,NaN,0.177419,NaN,NaN,NaN,NaN,0.036336,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.123077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.002053,0.068746,0.636364,-0.195140,-0.233815,-0.096698,0.100088,NaN,NaN
2021-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.001025,-0.033076,0.001111,-0.145283,-0.096180,-0.096606,0.021548,NaN,NaN
2021-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.001026,0.038516,0.054384,-0.261589,-0.184983,-0.167630,-0.221094,NaN,NaN


In [21]:
feat_comb_df = pd.read_csv('/content/drive/MyDrive/11-785/Project/features_full_final.csv').set_index(['tic','rdq']).drop(['FYEARQ','FQTR'],axis=1)
feat_comb_df = feat_comb_df.dropna(axis=0,how='all')
feat_comb_df = feat_comb_df.dropna(axis=1,how='all')
feat_comb_df = feat_comb_df[sorted(feat_comb_df.columns)]
feat_comb_df

A2ME        AC        AT       ATO      BEME      BETA  \
tic   rdq                                                                      
0081A 1996-12-31       NaN       NaN       NaN       NaN       NaN       NaN   
      1997-01-31       NaN       NaN       NaN       NaN       NaN       NaN   
      1997-02-28  0.564765       NaN   140.274       NaN       NaN       NaN   
      1997-03-31  0.564765       NaN   140.274       NaN       NaN       NaN   
      1997-04-30  0.529462       NaN   146.767       NaN       NaN       NaN   
...                    ...       ...       ...       ...       ...       ...   
ZZ    2012-10-31  5.840417  0.377852   971.509  0.888069 -0.316438  0.503049   
      2012-11-30  5.840417  0.377852   971.509  0.888069 -0.316438  0.503035   
      2012-12-31  5.840417  0.377852   971.509  0.888069 -0.316438  0.500307   
      2013-01-31  4.440987  0.489857  1005.345  1.081163 -0.253677  0.501021   
      2013-02-28  4.440987  0.489857  1005.345  1.081163 -0.253677  0.500922   

                         C  CF      CF2P       CTO  ...  REL2HIGH  RESID_VAR  \
tic   rdq                                           ...                        
0081A 1996-12-31       NaN NaN       NaN       NaN  ...       NaN        NaN   
      1997-01-31       NaN NaN       NaN       NaN  ...       NaN        NaN   
      1997-02-28  0.011328 NaN       NaN       NaN  ...       NaN   1.896184   
      1997-03-31  0.011328 NaN       NaN       NaN  ...       NaN   2.542292   
      1997-04-30  0.014431 NaN       NaN       NaN  ...       NaN   2.653953   
...                    ...  ..       ...       ...  ...       ...        ...   
ZZ    2012-10-31  0.091438 NaN -0.015174  0.549053  ...  0.910204   1.851362   
      2012-11-30  0.091438 NaN -0.015174  0.549053  ...  0.885714   0.619041   
      2012-12-31  0.091438 NaN -0.015174  0.549053  ...  0.885714   0.598993   
      2013-01-31  0.127473 NaN  0.042809  0.614895  ...  0.881633   0.542592   
      2013-02-28  0.127473 NaN  0.042809  0.614895  ...  0.889796   0.429795   

                       RNA       ROA       ROE       S2P     SGA2S   ST_REV  \
tic   rdq                                                                     
0081A 1996-12-31       NaN       NaN       NaN       NaN       NaN      NaN   
      1997-01-31       NaN       NaN       NaN       NaN       NaN      NaN   
      1997-02-28       NaN       NaN       NaN       NaN       NaN   1.5583   
      1997-03-31       NaN       NaN       NaN       NaN       NaN  10.7738   
      1997-04-30       NaN       NaN       NaN       NaN       NaN  -2.7778   
...                    ...       ...       ...       ...       ...      ...   
ZZ    2012-10-31  0.118627 -0.009920  0.167075  7.568810  0.818410  36.2500   
      2012-11-30  0.118627 -0.009920  0.167075  7.568810  0.818410   2.2936   
      2012-12-31  0.118627 -0.009920  0.167075  7.568810  0.818410  -2.6906   
      2013-01-31  0.167337  0.002145 -0.024645  5.954049  0.798554   0.0000   
      2013-02-28  0.167337  0.002145 -0.024645  5.954049  0.798554  -0.4608   

                       SUV  VARIANCE  
tic   rdq                             
0081A 1996-12-31       NaN       NaN  
      1997-01-31       NaN       NaN  
      1997-02-28       NaN  4.225542  
      1997-03-31       NaN  6.975184  
      1997-04-30       NaN  7.456677  
...                    ...       ...  
ZZ    2012-10-31  0.669890  4.256721  
      2012-11-30 -0.125130  0.458271  
      2012-12-31  0.062727  0.420686  
      2013-01-31 -1.501970  0.311550  
      2013-02-28 -0.227316  0.230933  

[3689185 rows x 45 columns]

In [22]:
MIN_AVAILABLE_FEATURES = feat_comb_df.shape[1]//2

full_df_lst = []
for date in tqdm(date_idx):
    sub = feat_comb_df.loc[feat_comb_df.index.get_level_values(1) < date]
    sub = sub.groupby(sub.index.get_level_values('tic')).last(1)
    sub = sub.transpose()
    
    sub = sub.rank(pct=True,axis=1)*2-1
    idx = (sub.isna().sum(axis=0) >= MIN_AVAILABLE_FEATURES).index
    sub = sub[idx].fillna(0)
    
#     sub = sub.fillna(sub.median(axis=1),axis=0)
    
    sub['date'] = date
    sub =  sub.set_index(['date',sub.index])
    
    full_df_lst.append(sub)

100%|██████████| 506/506 [10:52<00:00,  1.29s/it]


In [23]:
del(feat_comb_df)
gc.collect()

52

In [24]:
orig = (83,360)
valid = (360,420)
test = (420,505)

In [25]:
class AEDataSet(torch.utils.data.Dataset):
    def __init__(self, macro_df, full_df_lst, returns_df, date_idx, start_i, end_i, rhs_input_assets=False,lag=1):
        self.full_df_lst = full_df_lst
        self.macro_df_idx = macro_df.index
        self.macro_df = macro_df.to_numpy()
        self.macro_df[np.isnan(self.macro_df)] = 0
        self.returns_df = returns_df
        self.date_idx = date_idx
        self.start_i, self.end_i = start_i, end_i
        self.rhs_input_assets = rhs_input_assets
        self.epsilon=1e-5
        self.lag=lag
        
        self.output_date_idx = date_idx[start_i+lag:end_i]
        
    def __len__(self):
        return self.end_i - self.start_i - self.lag
    
    def __getitem__(self, idx):
        input_idx = idx
        idx += self.start_i+self.lag
        date = self.date_idx[idx] # returns date
        
        macro = self.macro_df
        
        mask = ~np.isnan(self.returns_df.loc[date].to_numpy())
        traded_assets_idx = self.returns_df.columns[mask]
        
        lhs = self.full_df_lst[idx - self.lag]
        lhs_idx = lhs.columns
        
        comb_idx = traded_assets_idx.intersection(lhs_idx)
        
        lhs = lhs.reindex(comb_idx,axis=1,fill_value=0).to_numpy()
        
        rhs_tmp = self.returns_df.loc[date,comb_idx]
        rhs_tmp = rhs_tmp.clip(rhs_tmp.quantile(0.01),rhs_tmp.quantile(0.99))
        output = rhs_tmp.to_numpy()

        lhs_tmp = np.concatenate([lhs,np.expand_dims(np.ones(lhs.shape[1]),0)],axis=0)
        
        if self.rhs_input_assets:
            rhs = output
        else:
            try:
#               
                # Causing singular error. Changing to pseudo inverse
                rhs = (np.linalg.pinv(lhs_tmp @ lhs_tmp.T) @ lhs_tmp) @ rhs_tmp
#               
            except Exception as e:
                print("idx:",input_idx)
                print(e)
#                 return self.full_df_lst[idx - self.lag].reindex(traded_assets_idx,axis=1,fill_value=0), rhs_tmp
                # return lhs,lhs_tmp,output
                raise e
#             rhs.index = lhs_tmp.index
#             rhs = rhs.reindex(self.alpha_names,fill_value=0).clip(-5e4,5e4)
#             rhs['equal'] = equal_weighted_portfolio
#             rhs = rhs.to_numpy()
        return macro,self.macro_df_idx,date,lhs,rhs,output,comb_idx
        
        
def collate_fn(batch):
    macro = batch[0][0]
    macro_df_idx = batch[0][1]
    dates = [i for _,_,i,_,_,_,_ in batch]
    dates = [np.where(macro_df_idx == np.max(macro_df_idx[macro_df_idx <= i]))[0][0] for i in dates]
    len_x = [len(output) for _,_,_,_,_,output,_ in batch]
    full_size = max(len_x)
#     print(full_size)
    
    
    batch_lhs = [nn.functional.pad(torch.Tensor(lhs),pad=(0,full_size-lhs.shape[1],0,0)) for macro,_,_,lhs,rhs,output,_ in batch]
    batch_rhs = [torch.Tensor(rhs) for macro,_,_,lhs,rhs,output,_ in batch]
    batch_output = [nn.functional.pad(torch.Tensor(output),pad=(0,full_size-len(output))) for macro,_,_,lhs,rhs,output,_ in batch]
    batch_assets = [idx for macro,_,_,lhs,rhs,output,idx in batch]

    
    return torch.from_numpy(macro), torch.tensor(dates),\
        torch.cat([x.unsqueeze(0) for x in batch_lhs]),\
        torch.cat([x.unsqueeze(0) for x in batch_rhs]),\
        torch.cat([x.unsqueeze(0) for x in batch_output]), torch.tensor(len_x), batch_assets

In [26]:
def buildListFromConfigs(config, inp_size, out_size, batch_norm=True, dropout=0.5, initialization=False, nonlinear=nn.SELU):
    layer_lst = []
    for i, v in enumerate(list(config)+[out_size]):
        if batch_norm and i > 0: #generally no batch norm before the model starts
            layer_lst.append(nn.LayerNorm(inp_size))
        if dropout > 1e-8 and i > 0:
            layer_lst.append(nn.Dropout(dropout))
        layer_lst.append(nn.Linear(inp_size, v, bias=True))
        if initialization:
            nn.init.kaiming_normal_(layer_lst[-1].weight,mode='fan_in',nonlinearity='linear')
        if i < len(config):
            layer_lst.append(nonlinear())
        inp_size = v
    return layer_lst, inp_size

class BuildLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout, proj_size=None):
        super().__init__()
        self.num_layers = num_layers
        self.dropout = nn.ModuleList([nn.Dropout(dropout) for i in range(num_layers)]) #nn.ModuleList([LockedDropout(dropout) for i in range(num_layers)])
        layers = []
        for i in range(num_layers):
            layers.append(nn.LSTMCell(input_size, hidden_size))
            input_size = hidden_size
        self.layers = nn.ModuleList(layers)
        self.final = nn.Linear(input_size, proj_size) if proj_size is not None and input_size != proj_size else nn.Identity()
    
    def forward(self, data, hidden=None):
        if hidden is None:
            hidden = [None for i in range(self.num_layers)]
        for i in range(self.num_layers):
            data = self.dropout[i](data)
            data, hidden[i] = self.layers[i](data, hidden[i])
        data = self.final(data)
        return data, hidden
        

class ConditionalAutoEncoder(nn.Module):
    # starting macro is the state of the last 13 months. For training, set it to None (equivalent to setting it to a torch of zeros).
    # However, since testing data is low, run the macro net on the past <attention_span> months and send that. First month's data is not used.
    # Requires a <attention_span>*<n_macro_size> torch vector in the same device as the model, where n_macro_size is the output of the n_hidden_macro net.
    def __init__(self, n_alphas, n_factors, n_macro, n_hidden_macro, n_hidden_lhs, n_hidden_rhs, attention_vs_lstm='attention', starting_macro = None, starting_rhs = None,
                    batch_norm_lhs=True, batch_norm_rhs=False, dropout_p=0, initialization=True, rhs_input_assets = False, rhs_activation = True,
                    bn_momentum=0.1, bn_track_running_stats = True, macro_attention_span=1, rhs_attention_span=1, num_lstm_layers=1):
        # n_assets can be assets or portfolios:
            # In both cases, we are interested in asset return or portfolio return on the RHS
            # LHS is the original factor 
        super(ConditionalAutoEncoder, self).__init__()

        self.n_alphas = n_alphas
        self.n_factors = n_factors
        self.n_hidden_lhs = list(n_hidden_lhs)
        self.n_hidden_rhs = list(n_hidden_rhs)
        self.n_macro = n_macro
        self.n_hidden_macro = n_hidden_macro
        self.lstm = attention_vs_lstm != 'attention'

        if attention_vs_lstm == 'attention':
            self.prev_macro = torch.zeros((macro_attention_span, n_factors), requires_grad=True) if starting_macro is None else starting_macro
            self.prev_rhs   = torch.zeros((rhs_attention_span, n_factors), requires_grad=True) if starting_rhs is None else starting_rhs
            self.macro_attention_span = macro_attention_span
            self.rhs_attention_span = rhs_attention_span
            #attention_lst = buildListFromConfigs(n_hidden_macro, n_macro+macro_attention_span*n_factors + rhs_attention_span*n_factors, n_factors, batch_norm=True, dropout=dropout_p, initialization=True, nonlinear=nn.SELU)
            attention_lst, _ = buildListFromConfigs(n_hidden_macro, n_macro+macro_attention_span*n_factors + rhs_attention_span*n_factors, n_factors, batch_norm=True, dropout=dropout_p, initialization=True, nonlinear=nn.SELU)
            self.macro = nn.Sequential(*attention_lst)
        else:
            attention_lst, _ = buildListFromConfigs(n_hidden_macro, n_factors, n_factors, batch_norm=True, dropout=dropout_p, initialization=True, nonlinear=nn.SELU)
            self.macro = nn.Sequential(*attention_lst) #nn.Identity()
            self.macro_lstm = nn.LSTM(n_macro, n_factors, num_lstm_layers, dropout=dropout_p) #BuildLSTM(n_factors, n_factors, num_lstm_layers)

        
        lhs_lst, _ = buildListFromConfigs(n_hidden_lhs, n_alphas, n_factors, batch_norm_lhs, dropout=dropout_p, initialization=True, nonlinear=nn.SELU)
        rhs_lst, _ = buildListFromConfigs(n_hidden_rhs, n_alphas+1, n_factors, batch_norm_rhs, dropout=dropout_p, initialization=True, nonlinear=nn.SELU if rhs_activation else nn.Identity)
        if rhs_activation:
            rhs_lst.append(nn.SELU()) 
        
        self.lhs = nn.Sequential(*lhs_lst)
        self.rhs = nn.Sequential(*rhs_lst)
    
    def run_through_lhs(self, data):
        #y_lhs_lst = []
        #for i in range(x_lhs.shape[2]):
        #    y_cur = self.lhs(x_lhs[:,:,i])
        #    y_cur = torch.unsqueeze(y_cur,2)
        #    y_lhs_lst.append(y_cur)
        #y_lhs = torch.cat(y_lhs_lst,dim=2)

        shape = data.shape
        data = data.permute(0, 2, 1).reshape(-1, shape[1])
        out = self.lhs(data)
        return out.reshape(shape[0], shape[2], -1).permute(0, 2, 1)
    
    def run_through_macro(self, x_macro, y_rhs, idx):
        if self.lstm:
            y_macro, hidden = self.macro_lstm(x_macro[None, :, :])
            y_macro = torch.sigmoid(y_macro)
            y_macro = self.macro(y_macro)[0, idx, :][:, :, None]
        else:
            # If using Attention, store prev_macro and prev_rhs outside like lstm
            macro = torch.cat((x_macro, self.prev_macro.reshape(1, -1), self.prev_rhs.reshape(1, -1)))
            y_macro = self.macro(macro)
            hidden = None
            self.prev_macro = torch.roll(self.prev_macro, -1, 0)
            self.prev_macro[-1] = y_macro
            self.prev_rhs = torch.roll(self.prev_rhs, -1, 0)
            self.prev_rhs[-1] = y_rhs
        return y_macro, hidden

    def forward(self, x_macro, macro_idx, x_lhs, x_rhs):
        y_lhs = self.run_through_lhs(x_lhs)
        y_rhs = self.rhs(x_rhs).unsqueeze(dim=2)
        self.last_rhs_factors = y_rhs
        y_macro, hidden = self.run_through_macro(x_macro, y_rhs, macro_idx)
        out = y_lhs*y_rhs*y_macro
        out = torch.sum(out, dim=1) #Double check this
        return out, y_rhs

    def forward_given_factors(self, x_macro, macro_idx, x_lhs, factors):
        y_lhs = self.run_through_lhs(x_lhs)
        y_rhs = factors
        y_macro, hidden = self.run_through_macro(x_macro, y_rhs, macro_idx)
        #out = torch.bmm(y_lhs.transpose(1,2), y_rhs).squeeze(2)
        out = torch.sum(y_lhs*y_rhs*y_macro, dim=1) #Double check this
        return out

In [27]:
from sklearn.metrics import mean_squared_error
@print_durations()
def test(args,model,device,test_loader,criterion):
    model.eval()
        
    output_lst = []
    data_lst = []

    for data_macro, idx_macro, data_lhs, data_rhs, data_output, data_x_lens, data_assets in test_loader:
        # print("Shapes:",data_lhs.shape, data_rhs.shape, data_output.shape, data_mask.shape)
        idx_macro = idx_macro.to(device)
        data_macro = data_macro.float().to(device)
        data_lhs = data_lhs.float().to(device)
        data_rhs = data_rhs.float().to(device)
        data_output = data_output.float().to(device)
        data_x_lens = data_x_lens.int().to(device)
        output,_ = model(x_macro = data_macro, macro_idx=idx_macro, x_lhs = data_lhs, x_rhs = data_rhs)

        data_mask = torch.arange(data_lhs.shape[2])[None, :].to(device) < data_x_lens[:, None]
        output_masked = torch.masked_select(output,data_mask)
        data_output_masked = torch.masked_select(data_output,data_mask)
        assert (len(output_masked) == torch.sum(data_x_lens))
        # print("Loss Inside:",criterion(output_masked,data_output_masked))

        output_lst.append(output_masked)
        data_lst.append(data_output_masked)
    
    full_output = torch.cat(output_lst,axis=0)
    full_data = torch.cat(data_lst,axis=0)

    return mean_squared_error(full_data.cpu().detach().numpy(), full_output.cpu().detach().numpy())

@print_durations()
def train(args, model, device, train_loader, optimizer, criterion, epoch, max_norm=None):
    model.train()
    
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    
    total_loss = 0
    total_reg_loss=0
    
    N = len(train_loader)
    
    for batch_idx, (data_macro, idx_macro, data_lhs, data_rhs, data_output, data_x_lens, data_assets) in enumerate(train_loader):

        optimizer.zero_grad()
        idx_macro = idx_macro.to(device)
        data_macro = data_macro.float().to(device)
        data_lhs = data_lhs.float().to(device)
        data_rhs = data_rhs.float().to(device)
        data_output = data_output.float().to(device)
        data_x_lens = data_x_lens.int().to(device)

        if max_norm: torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)
        output,_ = model(x_macro = data_macro, macro_idx=idx_macro, x_lhs = data_lhs, x_rhs = data_rhs)
        
        data_mask = torch.arange(data_lhs.shape[2])[None, :].to(device) < data_x_lens[:, None]

        output_masked = torch.masked_select(output,data_mask)
        data_output_masked = torch.masked_select(data_output,data_mask)

        loss_tmp = criterion(output_masked, data_output_masked)

        regularization_loss = 0
        for p in model.parameters():
            regularization_loss += torch.sum(abs(p))

        loss = loss_tmp + args['l1_lambda'] * regularization_loss
        
        loss.backward()
        total_loss += loss
        total_reg_loss += regularization_loss * args['l1_lambda'] 

        # print("Gradient:", loss.grad)
        optimizer.step()

        batch_bar.set_postfix(
            total_loss="{:.04f}".format(float(total_loss / (batch_idx + 1))),
            reg_loss = "{:.04f}".format(float(total_reg_loss / (batch_idx + 1))),
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        batch_bar.update()
        
    print(f"Epoch {epoch}, total_loss={float(total_loss / N):.04f}, reg_loss = {float(total_reg_loss / N):.04f}")

In [28]:
def run(args, n_alphas, full_df_lst, returns_df, date_idx, train_start,
        train_end, valid_start, valid_end, n_macro, n_window=36, batch_size=16, checkpoint_name=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ConditionalAutoEncoder(
        n_alphas = n_alphas,
        n_factors = args['n_factors'],
        n_macro = n_macro,
        n_hidden_macro = args['n_hidden_macro'],
        n_hidden_lhs = args['n_hidden_lhs'],
        n_hidden_rhs = args['n_hidden_rhs'],
        attention_vs_lstm='lstm',
        batch_norm_lhs = args['batch_norm_lhs'],
        batch_norm_rhs = args['batch_norm_rhs'],
        dropout_p=args['dropout_p'],
        initialization=args['initialization'],
        rhs_input_assets=args['rhs_input_assets'],
        rhs_activation = args['rhs_activation'],
        num_lstm_layers = args['num_lstm_layers']
    #     bn_track_running_stats=args['bn_track_running_stats'],
    #     bn_momentum=args['bn_momentum']
    )
    
    print(model)
    
    gc.collect()
    torch.cuda.empty_cache()
    
    train_data = AEDataSet(macro_df,full_df_lst,returns_df,date_idx,train_start,train_end)
    valid_data = AEDataSet(macro_df,full_df_lst,returns_df,date_idx,valid_start,valid_end)
    extended_valid_data= AEDataSet(macro_df,full_df_lst,returns_df,date_idx,valid_start-n_window,valid_end)

    train_loader = torch.utils.data.DataLoader(
        train_data,
        batch_size = batch_size,
        pin_memory=True,
        num_workers=0,
        shuffle=True,
        collate_fn = collate_fn
    )

    valid_loader = torch.utils.data.DataLoader(
        valid_data,
        batch_size = batch_size,
        pin_memory=True,
        num_workers=0,
        shuffle=False,
        collate_fn = collate_fn
    )
    
    extended_valid_loader = torch.utils.data.DataLoader(
        extended_valid_data,
        batch_size = batch_size,
        pin_memory=True,
        num_workers=0,
        shuffle=False,
        collate_fn = collate_fn
    )
    
    
    

    model = model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=args['lr'], weight_decay=args['l2_lambda'])
    criterion = torch.nn.MSELoss()

    train_mse = test(args,model,device,train_loader,criterion)
    valid_mse = test(args,model,device,valid_loader,criterion)
    train_mse_lst, valid_mse_lst = [train_mse], [valid_mse]

    # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=args['gamma'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min',0.5,patience=5,threshold=1e-3,cooldown=10)
    print("Pre-Train Train MSE:", train_mse)
    print("Pre-Train Valid MSE:", valid_mse)

    best_model = None
    for epoch in range(1, args['epoch']+1):
        log_dict = {}

        gc.collect()
        torch.cuda.empty_cache()

        train(args,model,device,train_loader,optimizer,criterion,epoch,max_norm=args['max_norm'])
        gc.collect()
        torch.cuda.empty_cache()

        log_dict['train_mse'] = test(args,model,device,train_loader,criterion)
        log_dict['valid_mse'] = test(args,model,device,valid_loader,criterion)
        log_dict['lr'] = scheduler.optimizer.state_dict()['param_groups'][0]['lr']

        gc.collect()
        torch.cuda.empty_cache()

        print("Train MSE:",log_dict['train_mse'])
        print("Valid MSE:",log_dict['valid_mse'])

        scheduler.step(log_dict['valid_mse'])
        print('Next Learning Rate:', scheduler.optimizer.state_dict()['param_groups'][0]['lr'])

        print(train_mse_lst)
        print(valid_mse_lst)
        if log_dict['valid_mse'] < np.min(valid_mse_lst):
            best_model = model
            torch.save(best_model, './best_model.model')
            # torch.save(model.state_dict(), args['MODEL_PATH']+"/model_epoch_"+str(epoch))

        train_mse_lst.append(log_dict['train_mse'])
        valid_mse_lst.append(log_dict['valid_mse'])

        if len(valid_mse_lst)>args['patience'] and log_dict['valid_mse'] > \
            np.min(valid_mse_lst[:-args['patience']]):
                break
    
    if checkpoint_name:
        torch.save({
#           'epoch': epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
#           'scheduler_state_dict': scheduler.state_dict()
          }, f'{checkpoint_name}.checkpoint')
    
    print("Training done. Evaluating...")
    
    output_lst = []
    data_lst = []
    factors_lst = []

    for data_macro, idx_macro, data_lhs, data_rhs, data_output, data_x_lens, data_assets in tqdm(extended_valid_loader):
        # print("Shapes:",data_lhs.shape, data_rhs.shape, data_output.shape, data_mask.shape)
        idx_macro = idx_macro.to(device)
        data_macro = data_macro.float().to(device)
        data_lhs = data_lhs.float().to(device)
        data_rhs = data_rhs.float().to(device)
        data_output = data_output.float().to(device)
        data_x_lens = data_x_lens.int().to(device)
        output, factors = best_model(x_macro = data_macro, macro_idx=idx_macro, x_lhs = data_lhs, x_rhs = data_rhs)

        data_mask = torch.arange(data_lhs.shape[2])[None, :].to(device) < data_x_lens[:, None]
        output_masked = torch.masked_select(output,data_mask)
        data_output_masked = torch.masked_select(data_output,data_mask)
        assert (len(output_masked) == torch.sum(data_x_lens))

        indices = [0]+list(torch.cumsum(data_x_lens,axis=0).cpu().numpy())

        for k,(i,j) in enumerate(zip(indices[:-1],indices[1:])):
            output_lst.append(pd.Series(output_masked[i:j].cpu().detach().numpy(),index=data_assets[k]))
            data_lst.append(pd.Series(data_output_masked[i:j].cpu().detach().numpy(),index=data_assets[k]))
            factors_lst.append(pd.Series(factors[k].squeeze().cpu().detach().numpy()))

    gc.collect()
    torch.cuda.empty_cache()
    
    output_df = pd.DataFrame(output_lst,index=extended_valid_data.output_date_idx).iloc[n_window:]
    data_output_df = pd.DataFrame(data_lst,index=extended_valid_data.output_date_idx).iloc[n_window:]
    factors_df = pd.DataFrame(factors_lst,index=extended_valid_data.output_date_idx)
    
    predicted_factors = factors_df.rolling(n_window).mean()
    factors_tensor = torch.tensor(predicted_factors.iloc[n_window:].to_numpy()).unsqueeze(2).float().to(device)
    
    
    pred_output_lst = []
    for i, (data_macro, idx_macro, data_lhs, _, _, data_x_lens, data_assets) in tqdm(enumerate(valid_loader),total=len(valid_loader)):
        idx_macro = idx_macro.to(device)
        data_macro = data_macro.float().to(device)
        data_lhs = data_lhs.float().to(device)
        data_x_lens = data_x_lens.int().to(device)
        output= best_model.forward_given_factors(data_macro, macro_idx=idx_macro, x_lhs = data_lhs, factors = factors_tensor[i*batch_size:(i+1)*batch_size])

        data_mask = torch.arange(data_lhs.shape[2])[None, :].to(device) < data_x_lens[:, None]
        output_masked = torch.masked_select(output,data_mask)
        assert (len(output_masked) == torch.sum(data_x_lens))

        indices = [0]+list(torch.cumsum(data_x_lens,axis=0).cpu().numpy())

        for k,(i,j) in enumerate(zip(indices[:-1],indices[1:])):
            pred_output_lst.append(pd.Series(output_masked[i:j].cpu().detach().numpy(),index=data_assets[k]))
    tm1_predicted_returns = pd.DataFrame(pred_output_lst,index=predicted_factors.iloc[n_window:].index)
    predicted_returns = tm1_predicted_returns.shift(1).iloc[1:]
    
    gc.collect()
    torch.cuda.empty_cache()
    
    log_dict = {}
    
#     log_dict['predicted_returns'] = predicted_returns
    
    # Predictive Stats:
    # Spearman Corr
    corr = predicted_returns.corrwith(returns_df.loc[predicted_returns.index,predicted_returns.columns],axis=1,method='spearman')
    log_dict['pred_spearman_corr'] = corr.mean()
    # R2
    numer = (returns_df.loc[predicted_returns.index,predicted_returns.columns]-predicted_returns)
    numer_mask = (~numer.isna()).astype(int)
    denom = returns_df.loc[predicted_returns.index,predicted_returns.columns] * numer_mask
    log_dict['pred_r2'] = (1 - ((numer**2).sum(axis=1))/(denom**2).sum(axis=1)).mean()
    
    # Attribution Stats:
    # Spearman Corr
    corr = output_df.corrwith(returns_df.loc[output_df.index,output_df.columns],axis=1,method='spearman')
    log_dict['total_spearman_corr'] = corr.mean()
    # R2
    numer = (returns_df.loc[output_df.index,output_df.columns]-output_df)
    numer_mask = (~numer.isna()).astype(int)
    denom = returns_df.loc[output_df.index,output_df.columns] * numer_mask
    log_dict['total_r2'] = (1 - ((numer**2).sum(axis=1))/(denom**2).sum(axis=1)).mean()
    
    # Trading Stats:
    train_pred_rank = predicted_returns.rank(pct=True,axis=1,numeric_only=True,ascending=True)
    long_port = ((train_pred_rank >= 0.9) & (train_pred_rank < 1)).astype(int)
    long_port = long_port.divide(long_port.sum(axis=1),axis=0)
    long_port_returns = (long_port*returns_df.loc[long_port.index]).sum(axis=1)
    short_port = (train_pred_rank <= 0.1).astype(int)
    short_port = short_port.divide(short_port.sum(axis=1),axis=0)
    short_port_returns = (short_port*returns_df.loc[short_port.index]).sum(axis=1)
    comb_port = long_port - short_port
    comb_port_returns = (comb_port*returns_df.loc[comb_port.index]).sum(axis=1)
    log_dict['long_short_SR'] = (comb_port_returns.mean()/comb_port_returns.std())*np.sqrt(12)
    log_dict['long_short_returns'] = comb_port_returns.mean()*12
    log_dict['long_short_SD'] = comb_port_returns.std()*np.sqrt(12)
    
    log_dict['long_SR'] = (long_port_returns.mean()/long_port_returns.std())*np.sqrt(12)
    log_dict['long_returns'] = long_port_returns.mean()*12
    log_dict['long_SD'] = long_port_returns.std()*np.sqrt(12)
    
    log_dict['short_SR'] = (short_port_returns.mean()/short_port_returns.std())*np.sqrt(12)
    log_dict['short_returns'] = short_port_returns.mean()*12
    log_dict['short_SD'] = short_port_returns.std()*np.sqrt(12)

    log_dict['model_state_dict'] = best_model.state_dict()
    
    return log_dict

In [29]:
fixed_args = {
#     'n_factors' : 5,
#     'n_hidden_lhs':[16,8],
    'n_hidden_rhs':[],
    'batch_norm_lhs' : True,
    'batch_norm_rhs' : False,
#     'rhs_activation': True,
    'dropout_p' : 0.0,
    'max_norm' : 1,
    'bn_momentum': 0.01,
    'bn_track_running_stats': True,
    'initialization' : False,
    'rhs_input_assets' : False,
    'lr' : 5e-4,
    'gamma': 1/4,
    'l2_lambda': 0,
    'l1_lambda': 4e-4,
    'batch_size': 32,
    'log_interval': 1,
    'epoch': 60,
    'patience': 6
}

search_args = {
    'n_factors': [4, 6, 8],
#     'n_factors': [4],
    'n_hidden_lhs': [(24,12)],
#     'n_hidden_lhs': [(16,8)],
    'rhs_activation': [False],
    'n_hidden_macro': [(16, 8), (8, 4), (16,), (8,), (4,)],
    'num_lstm_layers': [1, 2, 4, 8]
}

# fixed_args = {
# #     'n_factors' : 5,
# #     'n_hidden_lhs':[16,8],
#     'n_hidden_rhs':[],
#     'batch_norm_lhs' : True,
#     'batch_norm_rhs' : False,
#     'rhs_activation': True,
#     'dropout_p' : 0.0,
#     'max_norm' : 1,
#     'bn_momentum': 0.01,
#     'bn_track_running_stats': True,
#     'initialization' : False,
# #     'rhs_input_assets' : False,
#     'lr' : 5e-4,
#     'gamma': 1/4,
#     'l2_lambda': 0,
#     'l1_lambda': 4e-3,
#     'batch_size': 32,
#     'log_interval': 1,
#     'epoch': 3,
#     'patience': 6
# }

# search_args = {
#     'n_factors': [2],
#     'n_hidden_lhs': [(8,)],
#     'rhs_input_assets': [True]
# }

search_keys = list(search_args.keys())
search_values = list(search_args.values())
from itertools import product
search_values = list(product(*search_values))

In [ ]:
res_dict = {}

filepath = '/content/drive/MyDrive/11-785/Project/res_dict_midterm_corrected_sigm_lstm_later_low_reg.pickle'

if os.path.isfile(filepath):
    with open(filepath, 'rb') as handle:
        res_dict = pickle.load(handle)

for i,tpl in enumerate(search_values):
    print(f"Params {i}:", tpl)
    if i < len(res_dict):
        continue
    args = fixed_args.copy()
    for k,v in zip(search_keys,tpl):
        args[k] = v
            
    str_tpl = [str(tpl[0]),"_".join(list(map(str,tpl[1]))),"activation" if tpl[2] else "noactivation"]
    res_dict[tpl] = []
    display_dict = {}
    for j in range(10):
        log_dict = run(args,n_alphas=45, full_df_lst=full_df_lst, returns_df=returns_df, date_idx=date_idx, 
                    train_start=83, train_end=360, valid_start=360, valid_end=420, 
                    n_window=36, batch_size=16,
                        checkpoint_name=f'midterm_corrected_layers{str_tpl[0]}_hidden_{str_tpl[1]}_{str_tpl[2]}',
                    n_macro=147
    #                    checkpoint_name=f'midterm_corrected/layers{str_tpl[0]}_hidden_{str_tpl[1]}_{str_tpl[2]}'
                    )
        
        gc.collect()
        torch.cuda.empty_cache()
        
        res_dict[tpl].append(log_dict)
        for key, value in log_dict.items():
            if key != 'model_state_dict':
                display_dict[key] = display_dict.get(key, 0) + value

    for key, value in display_dict.items():
        print(key, value/10)
        
    with open(filepath, 'wb') as handle:
        pickle.dump(res_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

Params 0: (4, (24, 12), False, (16, 8), 1)
ConditionalAutoEncoder(
  (macro): Sequential(
    (0): Linear(in_features=4, out_features=16, bias=True)
    (1): SELU()
    (2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
    (3): Linear(in_features=16, out_features=8, bias=True)
    (4): SELU()
    (5): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
    (6): Linear(in_features=8, out_features=4, bias=True)
  )
  (macro_lstm): LSTM(147, 4)
  (lhs): Sequential(
    (0): Linear(in_features=45, out_features=24, bias=True)
    (1): SELU()
    (2): LayerNorm((24,), eps=1e-05, elementwise_affine=True)
    (3): Linear(in_features=24, out_features=12, bias=True)
    (4): SELU()
    (5): LayerNorm((12,), eps=1e-05, elementwise_affine=True)
    (6): Linear(in_features=12, out_features=4, bias=True)
  )
  (rhs): Sequential(
    (0): Linear(in_features=46, out_features=4, bias=True)
  )
)
   14.16 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cpu, <torch.utils.data.data

Epoch 1, total_loss=16.3504, reg_loss = 0.3733
   13.72 s in train({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cpu, <torch.utils.data.data..., AdamW ( Parameter Grou..., MSELoss(), 1, max_norm=1)
   11.51 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cpu, <torch.utils.data.data..., MSELoss())
    2.43 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cpu, <torch.utils.data.data..., MSELoss())
Train MSE: 7.2076015
Valid MSE: 0.028758543
Next Learning Rate: 0.0005
[30.542484]
[0.024948927]


Epoch 2, total_loss=5.3159, reg_loss = 0.3645
   13.48 s in train({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cpu, <torch.utils.data.data..., AdamW ( Parameter Grou..., MSELoss(), 2, max_norm=1)
   11.45 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cpu, <torch.utils.data.data..., MSELoss())
    2.42 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cpu, <torch.utils.data.data..., MSELoss())
Train MSE: 3.07209
Valid MSE: 0.026987515
Next Learning Rate: 0.0005
[30.542484, 7.2076015]
[0.024948927, 0.028758543]


Epoch 3, total_loss=2.7497, reg_loss = 0.3558
   13.50 s in train({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cpu, <torch.utils.data.data..., AdamW ( Parameter Grou..., MSELoss(), 3, max_norm=1)
   11.67 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cpu, <torch.utils.data.data..., MSELoss())
    2.46 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cpu, <torch.utils.data.data..., MSELoss())
Train MSE: 1.7984629
Valid MSE: 0.022355448
Next Learning Rate: 0.0005
[30.542484, 7.2076015, 3.07209]
[0.024948927, 0.028758543, 0.026987515]


Epoch 4, total_loss=1.8109, reg_loss = 0.3472
   13.68 s in train({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cpu, <torch.utils.data.data..., AdamW ( Parameter Grou..., MSELoss(), 4, max_norm=1)
   11.49 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cpu, <torch.utils.data.data..., MSELoss())
    2.44 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cpu, <torch.utils.data.data..., MSELoss())
Train MSE: 1.1965436
Valid MSE: 0.020131133
Next Learning Rate: 0.0005
[30.542484, 7.2076015, 3.07209, 1.7984629]
[0.024948927, 0.028758543, 0.026987515, 0.022355448]


...

Epoch 57, total_loss=0.2368, reg_loss = 0.2030
   14.88 s in train({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cpu, <torch.utils.data.data..., AdamW ( Parameter Grou..., MSELoss(), 57, max_norm=1)
   11.54 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cpu, <torch.utils.data.data..., MSELoss())
    2.47 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cpu, <torch.utils.data.data..., MSELoss())
Train MSE: 0.032154467
Valid MSE: 0.011577173
Next Learning Rate: 6.25e-05
[73.83932, 12.960795, 3.1374352, 1.1174077, 0.49835095, 0.28028587, 0.17696483, 0.10844574, 0.07830851, 0.06336711, 0.055468187, 0.050893925, 0.048141923, 0.046163116, 0.044683263, 0.043243606, 0.042131517, 0.04125804, 0.040366653, 0.03961087, 0.038949598, 0.038592428, 0.0382515, 0.03792178, 0.037609786, 0.03726939, 0.036946945, 0.036673285, 0.03638105, 0.036037512, 0.03578414, 0.035516072, 0.03517141, 0.034905065, 0.0346667, 0.034396686, 0.03413168, 0.034013834, 0.0339069, 0.03378

Epoch 58, total_loss=0.2346, reg_loss = 0.2023
   14.78 s in train({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cpu, <torch.utils.data.data..., AdamW ( Parameter Grou..., MSELoss(), 58, max_norm=1)
   11.84 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cpu, <torch.utils.data.data..., MSELoss())
    2.48 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cpu, <torch.utils.data.data..., MSELoss())
Train MSE: 0.032094754
Valid MSE: 0.011576988
Next Learning Rate: 6.25e-05
[73.83932, 12.960795, 3.1374352, 1.1174077, 0.49835095, 0.28028587, 0.17696483, 0.10844574, 0.07830851, 0.06336711, 0.055468187, 0.050893925, 0.048141923, 0.046163116, 0.044683263, 0.043243606, 0.042131517, 0.04125804, 0.040366653, 0.03961087, 0.038949598, 0.038592428, 0.0382515, 0.03792178, 0.037609786, 0.03726939, 0.036946945, 0.036673285, 0.03638105, 0.036037512, 0.03578414, 0.035516072, 0.03517141, 0.034905065, 0.0346667, 0.034396686, 0.03413168, 0.034013834, 0.0339069, 0.03378

Epoch 59, total_loss=0.2334, reg_loss = 0.2016
   14.57 s in train({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cpu, <torch.utils.data.data..., AdamW ( Parameter Grou..., MSELoss(), 59, max_norm=1)
   11.72 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cpu, <torch.utils.data.data..., MSELoss())
    2.46 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cpu, <torch.utils.data.data..., MSELoss())
Train MSE: 0.03204061
Valid MSE: 0.0115769785
Next Learning Rate: 6.25e-05
[73.83932, 12.960795, 3.1374352, 1.1174077, 0.49835095, 0.28028587, 0.17696483, 0.10844574, 0.07830851, 0.06336711, 0.055468187, 0.050893925, 0.048141923, 0.046163116, 0.044683263, 0.043243606, 0.042131517, 0.04125804, 0.040366653, 0.03961087, 0.038949598, 0.038592428, 0.0382515, 0.03792178, 0.037609786, 0.03726939, 0.036946945, 0.036673285, 0.03638105, 0.036037512, 0.03578414, 0.035516072, 0.03517141, 0.034905065, 0.0346667, 0.034396686, 0.03413168, 0.034013834, 0.0339069, 0.03378

Epoch 60, total_loss=0.2335, reg_loss = 0.2009
   14.59 s in train({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cpu, <torch.utils.data.data..., AdamW ( Parameter Grou..., MSELoss(), 60, max_norm=1)
   11.54 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cpu, <torch.utils.data.data..., MSELoss())
    2.66 s in test({'n_hidden_rhs': [], '..., ConditionalAutoEncoder..., cpu, <torch.utils.data.data..., MSELoss())
Train MSE: 0.031990454
Valid MSE: 0.011576922
Next Learning Rate: 6.25e-05
[73.83932, 12.960795, 3.1374352, 1.1174077, 0.49835095, 0.28028587, 0.17696483, 0.10844574, 0.07830851, 0.06336711, 0.055468187, 0.050893925, 0.048141923, 0.046163116, 0.044683263, 0.043243606, 0.042131517, 0.04125804, 0.040366653, 0.03961087, 0.038949598, 0.038592428, 0.0382515, 0.03792178, 0.037609786, 0.03726939, 0.036946945, 0.036673285, 0.03638105, 0.036037512, 0.03578414, 0.035516072, 0.03517141, 0.034905065, 0.0346667, 0.034396686, 0.03413168, 0.034013834, 0.0339069, 0.03378

100%|██████████| 4/4 [00:02<00:00,  1.57it/s]


In [ ]:
res_dict